##  Python script to clean ridership and graphhopper data for analysis

### 1. Import required packages

In [2]:
import pandas as pd
import geopandas as gpd

### 2. Read data for cleaning

In [3]:
# June 2024 bike share ridership data
df_june2024_ridership = pd.read_csv('data/june2024bikeshareridership-utf8.csv')

In [4]:
# Graphhopper calculation data for distance and elevation delta estimations
df_graphhopper = pd.read_csv('data/trips-graphhopper-output-all.csv')

In [5]:
# Station coordinates 
station_coords = gpd.read_file('data/station-information-july112024.geojson')

# Extract latitude and longitude from 'geometry' column
station_coords['lat'] = station_coords.geometry.y
station_coords['lon'] = station_coords.geometry.x

# Drop the 'geometry' column
station_coords = station_coords.drop(columns=['geometry'])

# Assuming station_coords is your DataFrame
station_coords.head()

,station_id,lat,lon
0,7000,43.639832,-79.395954
1,7001,43.664964,-79.383550
2,7002,43.667333,-79.399429
3,7003,43.667158,-79.402761
4,7005,43.648001,-79.383177


### 3. Clean up data formatting for consistency

In [6]:
# Replace "EFIT G5" with "EFIT"
df_june2024_ridership.replace("EFIT G5", "EFIT", inplace=True)

# Rename start station and end station columns to match graphhopper data
df_june2024_ridership.rename(columns={
    'Start Station Id': 'start_id',
    'End Station Id': 'end_id'
}, inplace=True)

# Reformat all headers for consistency
df_june2024_ridership.columns = df_june2024_ridership.columns.str.lower().str.replace(' ', '_')

### 4. Join additional data for average distance and average elevation delta from graphhopper to ridership

In [7]:
# Group by 'start_id' and 'end_id' and calculate the average for 'duration', 'distance', and 'elevation_delta'
average_df = df_graphhopper.groupby(['start_id', 'end_id']).agg({
    'distance': 'mean',
    'elevation_delta': 'mean'
}).reset_index()

# Rename the columns for clarity
average_df.rename(columns={
    'distance': 'distance_average',
    'elevation_delta': 'elevation_delta_average'
}, inplace=True)

# Merge the average_df with ridership_df on 'start_id' and 'end_id'
merged_df = df_june2024_ridership.merge(average_df, on=['start_id', 'end_id'], how='left')
merged_df.head()

,id,closed_status,duration,start_id,start_date,start_station_name,end_date,end_id,end_station_name,rider_type,bike_barcode,bike_public_identifier,bike_qrcode,bike_model,trip_origin,product_name,distance_average,elevation_delta_average
0,29681508,NORMAL,62,7948,6/30/2024 3:16,Bathurst St / Housey St,6/30/2024 3:17,7948,Bathurst St / Housey St,MEMBER,P26116,10095892,fd3c233d7c,ICONIC,MOBILE,Annual 30,4409.766,39.5
1,29682786,NORMAL,876,7948,6/30/2024 8:18,Bathurst St / Housey St,6/30/2024 8:32,7938,Portland St / Wellington St W,MEMBER,P38112,NaN,d5688ac2eb,ICONIC,KEYCARD,Annual 45,824.377,13.5
2,29644869,NORMAL,766,7948,6/28/2024 16:04,Bathurst St / Housey St,6/28/2024 16:16,7927,Strachan Ave / East Liberty St - SMART,MEMBER,P10462,6943229,69a1a12c1f,ICONIC,MOBILE,Annual 30,1566.971,12.5
3,29672434,NORMAL,1333,7948,6/29/2024 18:39,Bathurst St / Housey St,6/29/2024 19:02,7927,Strachan Ave / East Liberty St - SMART,MEMBER,P28806,13857485,5c69c0238e,ICONIC,MOBILE,Annual 30,1566.971,12.5
4,29691472,NORMAL,1002,7948,6/30/2024 14:07,Bathurst St / Housey St,6/30/2024 14:24,7802,King St W / Jameson Ave - SMART,MEMBER,P02087,6205539,34ad66dbbe,ICONIC,MOBILE,Annual 45,3689.160,27.5


### 5. Parse time data

In [8]:
# Convert 'start_date' and 'end_date' to datetime objects
merged_df['start_date'] = pd.to_datetime(merged_df['start_date'], format='%m/%d/%Y %H:%M')   
merged_df['end_date'] = pd.to_datetime(merged_df['end_date'], format='%m/%d/%Y %H:%M')

# Extract date and time components
merged_df['start_date_only'] = merged_df['start_date'].dt.date
merged_df['start_time_only'] = merged_df['start_date'].dt.time
merged_df['start_time_seconds'] = merged_df['start_date'].dt.hour * 3600 + merged_df['start_date'].dt.minute * 60 + merged_df['start_date'].dt.second
merged_df['end_date_only'] = merged_df['end_date'].dt.date
merged_df['end_time_only'] = merged_df['end_date'].dt.time
merged_df['end_time_seconds'] = merged_df['start_time_seconds']+merged_df['duration']

# Extract day of the week for 'start_date'
merged_df['day_of_week'] = merged_df['start_date'].dt.day_name()

merged_df.head()

,id,closed_status,duration,start_id,start_date,start_station_name,end_date,end_id,end_station_name,rider_type,...,product_name,distance_average,elevation_delta_average,start_date_only,start_time_only,start_time_seconds,end_date_only,end_time_only,end_time_seconds,day_of_week
0,29681508,NORMAL,62,7948,2024-06-30 03:16:00,Bathurst St / Housey St,2024-06-30 03:17:00,7948,Bathurst St / Housey St,MEMBER,...,Annual 30,4409.766,39.5,2024-06-30,03:16:00,11760,2024-06-30,03:17:00,11822,Sunday
1,29682786,NORMAL,876,7948,2024-06-30 08:18:00,Bathurst St / Housey St,2024-06-30 08:32:00,7938,Portland St / Wellington St W,MEMBER,...,Annual 45,824.377,13.5,2024-06-30,08:18:00,29880,2024-06-30,08:32:00,30756,Sunday
2,29644869,NORMAL,766,7948,2024-06-28 16:04:00,Bathurst St / Housey St,2024-06-28 16:16:00,7927,Strachan Ave / East Liberty St - SMART,MEMBER,...,Annual 30,1566.971,12.5,2024-06-28,16:04:00,57840,2024-06-28,16:16:00,58606,Friday
3,29672434,NORMAL,1333,7948,2024-06-29 18:39:00,Bathurst St / Housey St,2024-06-29 19:02:00,7927,Strachan Ave / East Liberty St - SMART,MEMBER,...,Annual 30,1566.971,12.5,2024-06-29,18:39:00,67140,2024-06-29,19:02:00,68473,Saturday
4,29691472,NORMAL,1002,7948,2024-06-30 14:07:00,Bathurst St / Housey St,2024-06-30 14:24:00,7802,King St W / Jameson Ave - SMART,MEMBER,...,Annual 45,3689.160,27.5,2024-06-30,14:07:00,50820,2024-06-30,14:24:00,51822,Sunday


### 6. Add start and end station coordinates

In [9]:
# Convert both columns to the same data type
merged_df['start_id'] = merged_df['start_id'].astype(int)
merged_df['end_id'] = merged_df['end_id'].astype(int)
station_coords['station_id'] = station_coords['station_id'].astype(int)

# Perform the merge operation for start_id
merged_df = pd.merge(merged_df, station_coords, left_on='start_id', right_on='station_id', how='left')

# Rename the lat and lon columns for start_id
merged_df.rename(columns={'lat': 'start_lat', 'lon': 'start_lon'}, inplace=True)

# Drop the station_id column
merged_df.drop(columns=['station_id'], inplace=True)

# Perform the merge operation for end_id
merged_df = pd.merge(merged_df, station_coords, left_on='end_id', right_on='station_id', how='left', suffixes=('', '_end'))

# Rename the lat and lon columns for end_id
merged_df.rename(columns={'lat': 'end_lat', 'lon': 'end_lon'}, inplace=True)

# Drop the station_id column from the second merge
merged_df.drop(columns=['station_id'], inplace=True)

# Display the result
merged_df.head()

,id,closed_status,duration,start_id,start_date,start_station_name,end_date,end_id,end_station_name,rider_type,...,start_time_only,start_time_seconds,end_date_only,end_time_only,end_time_seconds,day_of_week,start_lat,start_lon,end_lat,end_lon
0,29681508,NORMAL,62,7948,2024-06-30 03:16:00,Bathurst St / Housey St,2024-06-30 03:17:00,7948,Bathurst St / Housey St,MEMBER,...,03:16:00,11760,2024-06-30,03:17:00,11822,Sunday,43.637819,-79.400132,43.637819,-79.400132
1,29682786,NORMAL,876,7948,2024-06-30 08:18:00,Bathurst St / Housey St,2024-06-30 08:32:00,7938,Portland St / Wellington St W,MEMBER,...,08:18:00,29880,2024-06-30,08:32:00,30756,Sunday,43.637819,-79.400132,43.642902,-79.399370
2,29644869,NORMAL,766,7948,2024-06-28 16:04:00,Bathurst St / Housey St,2024-06-28 16:16:00,7927,Strachan Ave / East Liberty St - SMART,MEMBER,...,16:04:00,57840,2024-06-28,16:16:00,58606,Friday,43.637819,-79.400132,43.639065,-79.410810
3,29672434,NORMAL,1333,7948,2024-06-29 18:39:00,Bathurst St / Housey St,2024-06-29 19:02:00,7927,Strachan Ave / East Liberty St - SMART,MEMBER,...,18:39:00,67140,2024-06-29,19:02:00,68473,Saturday,43.637819,-79.400132,43.639065,-79.410810
4,29691472,NORMAL,1002,7948,2024-06-30 14:07:00,Bathurst St / Housey St,2024-06-30 14:24:00,7802,King St W / Jameson Ave - SMART,MEMBER,...,14:07:00,50820,2024-06-30,14:24:00,51822,Sunday,43.637819,-79.400132,43.637358,-79.436180


### 6.5. Drop columns that won't be used in analysis

In [10]:
# Drop the specified columns
merged_df = merged_df.drop(columns=['bike_barcode', 'bike_public_identifier', 'bike_qrcode', 'product_name'])

### 7. Write file to data/ridership.csv for analysis

In [11]:
# merged_df.to_csv('data/ridership.csv', index=False)
merged_df.head()

,id,closed_status,duration,start_id,start_date,start_station_name,end_date,end_id,end_station_name,rider_type,...,start_time_only,start_time_seconds,end_date_only,end_time_only,end_time_seconds,day_of_week,start_lat,start_lon,end_lat,end_lon
0,29681508,NORMAL,62,7948,2024-06-30 03:16:00,Bathurst St / Housey St,2024-06-30 03:17:00,7948,Bathurst St / Housey St,MEMBER,...,03:16:00,11760,2024-06-30,03:17:00,11822,Sunday,43.637819,-79.400132,43.637819,-79.400132
1,29682786,NORMAL,876,7948,2024-06-30 08:18:00,Bathurst St / Housey St,2024-06-30 08:32:00,7938,Portland St / Wellington St W,MEMBER,...,08:18:00,29880,2024-06-30,08:32:00,30756,Sunday,43.637819,-79.400132,43.642902,-79.399370
2,29644869,NORMAL,766,7948,2024-06-28 16:04:00,Bathurst St / Housey St,2024-06-28 16:16:00,7927,Strachan Ave / East Liberty St - SMART,MEMBER,...,16:04:00,57840,2024-06-28,16:16:00,58606,Friday,43.637819,-79.400132,43.639065,-79.410810
3,29672434,NORMAL,1333,7948,2024-06-29 18:39:00,Bathurst St / Housey St,2024-06-29 19:02:00,7927,Strachan Ave / East Liberty St - SMART,MEMBER,...,18:39:00,67140,2024-06-29,19:02:00,68473,Saturday,43.637819,-79.400132,43.639065,-79.410810
4,29691472,NORMAL,1002,7948,2024-06-30 14:07:00,Bathurst St / Housey St,2024-06-30 14:24:00,7802,King St W / Jameson Ave - SMART,MEMBER,...,14:07:00,50820,2024-06-30,14:24:00,51822,Sunday,43.637819,-79.400132,43.637358,-79.436180
